# Face_mask_detection_CNN_OpenCV

#### Author : Amin  4.03.2023

#### Aim of the Project
The dataset consists of 1376 images consisting of two classes– with_mask and without_mask. The objective to create a Face Mask Detector using CNN and OpenCV.

**Since we have the data set , we have to generate our pre-trained model: 'facemask_cnn_model.h5'**

Pre-trained models are pre-built neural network models that have already been trained on large datasets. These models have learned to recognize patterns and features in the data that they have been trained on. Pre-trained models can be used as a starting point for building custom models for specific tasks, which can save time and computational resources.
In the case of the face mask detection model, the pre-trained model can recognize certain patterns and features in images that indicate the presence or absence of a face mask. Instead of starting from scratch, we can use a pre-trained model as a starting point and fine-tune it on our own dataset, which can lead to better performance and faster training times. Once the pre-trained model has been fine-tuned on our dataset, it can be saved as a .h5 file and used for face mask detection in real-time.

The **haarcascade_frontalface_default.xml** file contains pre-trained models for detecting human faces in images and video streams. The file is part of the OpenCV library and is based on the Haar Cascade Classifier, which is a machine learning-based approach for object detection.
In the context of the face mask detection model, we need to detect faces in real-time video streams so that we can determine whether a person is wearing a face mask or not. We use the haarcascade_frontalface_default.xml file along with the OpenCV library to detect faces in the video stream. Once we have detected a face, we can then pass the image of the face to the pre-trained model to determine whether the person is wearing a face mask or not.
In summary, the haarcascade_frontalface_default.xml file is an important component of the face mask detection model as it allows us to detect human faces in real-time video streams.

## Import packages

• The first line imports the OpenCV library for image processing.

• The second line imports the NumPy library for numerical operations.

• The third line imports the Sequential class from the Keras library, which is used to create deep learning models.

• The fourth line imports the Conv2D, MaxPooling2D, Flatten, and Dense classes from Keras, which are used to create layers in the model.

• The fifth line imports ModelCheckpoint and LearningRateScheduler classes from Keras callbacks module, which are used to define custom behaviour during model training.

• The sixth line imports train_test_split function from sklearn.model_selection module, which is used to split dataset into train and test set.

• The seventh line imports math library for mathematical operations.

• The eighth line imports os module which provides a way of using operating system dependent functionality.

• The ninth line keras.models.load_model is a function that allows you to load a pre-trained Keras model from a file. It is used when you want to use a previously trained model for inference or further training.

In [1]:
import os 
import cv2
import math
import numpy as np



from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import train_test_split




import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Ignore specific warning
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


* **These lines define constants to be used in the model. IMG_SIZE defines the size of the image, BATCH_SIZE defines the number of images to be processed in each batch, NUM_EPOCHS defines the number of epochs to train the model for, and MODEL_FILE is the filename to save the trained model.**

In [2]:
# Define constants
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 10
MODEL_FILE = 'facemask_cnn_model.h5'

* **This function defines a learning rate schedule that will be used during model training. The learning rate is set to 1e-3 initially, but after 5 epochs it is halved.**

In [3]:
# Define learning rate schedule
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 5:
        lr *= 0.5
    return lr


* **These lines load the image data from the directories 'with_mask' and 'without_mask' using OpenCV, resize them to the IMG_SIZE, and store them in NumPy arrays. Then the arrays are concatenated to create the full dataset, and split into training and testing sets using train_test_split function.**

In [4]:
# Load data
mask_images = np.array([cv2.resize(cv2.imread('with_mask/' + filename), (IMG_SIZE, IMG_SIZE)) for filename in os.listdir('with_mask')])
nomask_images = np.array([cv2.resize(cv2.imread('without_mask/' + filename), (IMG_SIZE, IMG_SIZE)) for filename in os.listdir('without_mask')])
X = np.concatenate((mask_images, nomask_images))
y = np.concatenate((np.ones(len(mask_images)), np.zeros(len(nomask_images))))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* **The Sequential class is a linear stack of layers that allows you to build a deep learning model. This line initializes an instance of the Sequential class and assigns it to the variable model.**

In [5]:
# Create CNN model
model = Sequential()

* **This adds a convolutional layer with 32 filters, each of size 3x3, and the ReLU activation function. The input_shape argument specifies the shape of the input to the layer, which is a 3D tensor of size (IMG_SIZE, IMG_SIZE, 3), where the last dimension represents the number of color channels (RGB).**

In [6]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))

* **This adds a max pooling layer with a pool size of 2x2, which reduces the spatial dimensions of the previous layer by a factor of 2.**

In [7]:
model.add(MaxPooling2D((2, 2)))

* **These lines add additional convolutional and max pooling layers, gradually increasing the number of filters and decreasing the spatial dimensions of the output.**

In [8]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

* **This flattens the output from the previous layer into a 1D array, which can be fed into a fully connected layer.**

In [9]:
model.add(Flatten())

* **This adds a fully connected layer with 256 neurons and the ReLU activation function.**

In [10]:
model.add(Dense(256, activation='relu'))

* **This adds the output layer with a single neuron and the sigmoid activation function, which outputs a value between 0 and 1 representing the probability that the input belongs to the positive class (wearing a mask).**

In [11]:
model.add(Dense(1, activation='sigmoid'))

* **This compiles the model by specifying the optimizer to use (Adam), the loss function to optimize (binary cross-entropy), and the evaluation metrics to use (accuracy).**

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

* **These lines define two callback functions to be used during training. ModelCheckpoint saves the model with the highest validation accuracy during training, and LearningRateScheduler adjusts the learning rate according to a predefined schedule.**

In [13]:
#Define callbacks
checkpoint = ModelCheckpoint(MODEL_FILE, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
lr_scheduler = LearningRateScheduler(lr_schedule)

* **This fits the model to the training data (X_train and y_train) for a specified number of epochs (NUM_EPOCHS) and batch size (BATCH_SIZE), using the validation data (X_test and y_test) to monitor the performance of the model during training. The callbacks argument specifies the callback functions to be used during training, including ModelCheckpoint and LearningRateScheduler.**

In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, lr_scheduler])

Epoch 1/10
35/35 [==============================] - ETA: 0s - loss: 30.8564 - accuracy: 0.6673
Epoch 1: val_accuracy improved from -inf to 0.82246, saving model to facemask_cnn_model.h5
35/35 [==============================] - 102s 3s/step - loss: 30.8564 - accuracy: 0.6673 - val_loss: 0.3923 - val_accuracy: 0.8225 - lr: 0.0010
Epoch 2/10
35/35 [==============================] - ETA: 0s - loss: 0.3275 - accuracy: 0.8836
Epoch 2: val_accuracy improved from 0.82246 to 0.84058, saving model to facemask_cnn_model.h5
35/35 [==============================] - 101s 3s/step - loss: 0.3275 - accuracy: 0.8836 - val_loss: 0.3565 - val_accuracy: 0.8406 - lr: 0.0010
Epoch 3/10
35/35 [==============================] - ETA: 0s - loss: 0.1312 - accuracy: 0.9600
Epoch 3: val_accuracy improved from 0.84058 to 0.93478, saving model to facemask_cnn_model.h5
35/35 [==============================] - 99s 3s/step - loss: 0.1312 - accuracy: 0.9600 - val_loss: 0.1848 - val_accuracy: 0.9348 - lr: 0.0010
Epoch 4/1

* **Then we only need to save the pretained model in order to use it for the real time.** 

In [15]:
# Save the model
model.save(MODEL_FILE)

### The code loads a pre-trained convolutional neural network (CNN) model from a saved file and uses it to classify faces in the video frames. 

### The code then opens a video capture device and continuously loops through the frames, calling the detect_face_mask function to process each frame and displaying the result in a window.

* **This line sets the IMG_SIZE constant to 224, which is the expected input size of the pre-trained CNN model.**
* **The next line sets the MODEL_FILE constant to the name of the file containing the pre-trained CNN model.**

In [16]:
# Define constants
IMG_SIZE = 224
MODEL_FILE = 'facemask_cnn_model.h5'

* **This line loads the pre-trained CNN model from the MODEL_FILE file using the load_model function from the Keras library.**

In [17]:
# Load the trained model
model = load_model(MODEL_FILE)

* **This code defines a function called detect_face_mask that takes a single argument frame, which is a frame from a video or webcam feed. The purpose of the function is to detect whether or not faces in the frame are wearing masks.**


* **The first step of the function is to convert the input frame from a color image to a grayscale image using the OpenCV function cv2.cvtColor.**


* **Next, the function uses a Haar cascade classifier to detect faces in the grayscale image. The classifier is initialized with the cv2.CascadeClassifier function, which loads a pre-trained XML file containing the classifier data. The detectMultiScale function is then called on the grayscale image to detect faces, with several parameters such as scaleFactor, minNeighbors, and minSize specified to control the sensitivity and accuracy of the detection.**


* **For each face detected, the function extracts the region of interest (ROI) from the original color image, resizes it to a specified size using cv2.resize, and preprocesses the ROI for input to a machine learning model. The np.array function is used to convert the ROI to a NumPy array, which is then divided by 255.0 to normalize the pixel values to the range [0, 1]. The np.expand_dims function is used to add an extra dimension to the array to match the expected input shape of the model.**


* **The machine learning model is then used to predict whether or not the face in the ROI is wearing a mask. The prediction is obtained by calling the model.predict function on the preprocessed ROI, and indexing the result to retrieve the actual prediction value.**


* **Finally, the function draws a rectangle around the face in the original color image using cv2.rectangle, with the color of the rectangle determined by the prediction result. The label of the prediction is also written above the rectangle using cv2.putText, with the same color as the rectangle. The function returns the original color image with the rectangles and labels drawn.**

In [18]:
# Create a function to detect face masks in a frame
def detect_face_mask(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Use a Haar cascade classifier to detect faces in the grayscale image
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    # For each face detected, classify whether or not it is wearing a mask
    for (x, y, w, h) in faces:
        # Extract the face region from the frame and resize it to the input size of the model
        face_img = frame[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))

        # Preprocess the face image for input to the model
        face_array = np.array(face_img) / 255.0
        face_array = np.expand_dims(face_array, axis=0)

        # Use the model to predict whether or not the face is wearing a mask
        prediction = model.predict(face_array)[0][0]

        # Draw a rectangle around the face and label it with the prediction result
        label = 'Mask' if prediction > 0.5 else 'No mask'
        color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    return frame

* **This line initializes a video capture device with the index 0, which is usually the default camera on the computer.**

In [19]:
# Open a video capture device
cap = cv2.VideoCapture(0)

* **while True:: starts an infinite loop that will continuously process frames from the video capture device until the program is manually stopped or the device is disconnected.**


* **ret, frame = cap.read(): reads a single frame from the video capture device cap and stores it in the variable frame. The return value ret is a Boolean indicating whether the frame was read successfully or not.**


* **if not ret: break: checks whether the frame was read successfully. If it was not, the program exits the loop.**


* **frame = detect_face_mask(frame): calls the detect_face_mask() function to detect face masks in the current frame.**


* **cv2.imshow('Face Mask Detection', frame): displays the frame in a window named "Face Mask Detection".**


* **if cv2.waitKey(1) & 0xFF == ord('q'): break: waits for a key press and checks if the pressed key is the 'q' key. If it is, the program exits the loop.**


* **cap.release(): releases the video capture device, freeing up system resources.**


* **cv2.destroyAllWindows(): closes all windows opened by OpenCV.**

In [20]:
# Loop through frames from the video capture device
while True:
    # Read a frame from the video capture device
    ret, frame = cap.read()

    # If the frame was not read successfully, break out of the loop
    if not ret:
        break

    # Detect face masks in the frame
    frame = detect_face_mask(frame)

    # Display the frame in a window
    cv2.imshow('Face Mask Detection', frame)

    # Wait for a key press and check if the 'q' key was pressed to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture device and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 61ms/step


KeyboardInterrupt: 